# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”


First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [1]:
import os
os.chdir(str(os.getcwd())+"/../../")


from tasho import robot as rob
from tasho import discrete_plan as dp

from tasho import problem_template as pt
from tasho import environment as env

import casadi as cs
from rockit import MultipleShooting, Ocp
import numpy as np

## Instantiate plan

In [2]:
plan = dp.DiscretePlan()

## Set robot

In [3]:
# Initial joint configuration
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, 0]
# Initial joint velocity
q_dot0_val = [0]*7

robot = rob.Robot('kinova')
robot.set_joint_acceleration_limits(lb = -30*3.14159/180, ub = 30*3.14159/180)
robot.set_state(q0_val + q_dot0_val)
robot.set_robot_input_resolution("acceleration")

Loading robot params from json ...
Loaded 7-DoF robot: kinova


## Set environment

In [4]:
environment = env.Environment()

cube1 = env.Cube(length = 1, position = [0.5, 0, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/cube_small.urdf")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "/models/objects/table.urdf")
table2 = env.Box(height = 0.3, position = [0,0.5, 0], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/table.urdf")
# table3 = env.Box(height = 0.3, position = [0,-0.5, 0], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/table.urdf")

In [5]:
environment.add_object(cube1, "cube")
environment.add_object(table1, "table1")
environment.add_object(table2, "table2")
# environment.add_object(table3, "table3")

In [6]:
environment.print_objects()

['cube', 'table1', 'table2']


In [7]:
plan.add_environment(environment)

## Task 1: Approximation to object

In [8]:
horizon_size = 10
t_mpc = 0.5

T_goal = np.array([[0, 1, 0, 0.5], [1, 0, 0, 0], [0, 0, -1, 0.25], [0, 0, 0, 1]])  # T_goal = np.array([[0.0, 0., -1., 0.5], [0., 1., 0., 0.], [1.0, 0., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 0., -1., 0.5], [-1., 0., 0., 0.], [0., 1., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 1., 0., 0.5], [1., 0., 0., 0.], [0., 0., -1.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0, 1, 0, 0], [1, 0, 0, -0.5], [0, 0, -1, 0.5], [0, 0, 0, 1]])

approach_task = pt.Point2Point(horizon_size*t_mpc, horizon_steps = horizon_size, goal = T_goal)

approach_task.add_robot(robot)

Goal transformation matrix


## Task 2: Picking the object up

In [9]:
horizon_size = 30
t_mpc = 0.5

# T_goal = np.array([[-1, 0, 0, 0], [0, 1, 0, 0.5], [0, 0, -1, 0.25], [0, 0, 0, 1]])
# T_goal = np.array([[0, 0, 1, 0.5], [1, 0, 0, 0.2], [0, 1, 0, 0.5], [0, 0, 0, 1]])
T_goal = np.array([[0, 1, 0, 0.4], [1, 0, 0, 0.15], [0, 0, -1, 0.25], [0, 0, 0, 1]])

pickup_task = pt.Point2Point(time = horizon_size*t_mpc, horizon_steps = horizon_size, goal = T_goal)

pickup_task.add_robot(robot)

Goal transformation matrix


## Define discrete plan

In [10]:
plan.add_task(approach_task, "approach")
plan.add_task(pickup_task, "pickup")

In [12]:
plan.print_tasks()

['approach', 'pickup']


## Solve specific tasks in the plan

In [13]:
# sol = plan.solve_task(task_name = "approach", q_init = q0_val)
# sol_list = plan.solve_task(task_name = ["approach","pickup"], q_init = q0_val)
# sol_list = plan.execute_plan(q_init = q0_val)


## Simulate plan execution


In [14]:
robot_q0 = robot.get_initial_conditions[0:robot.ndof]
robot_qdot0 = robot.get_initial_conditions[robot.ndof:2*robot.ndof]

plan.simulate_plan(simulator = "bullet", q_init = robot_q0, qdot_init = robot_qdot0)

*********** THIS IS A CUBE ************
*********** THIS IS A BOX ************
*********** THIS IS A BOX ************
[2, 3, 4]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      530
Number of nonzeros in inequality constraint Jacobian.:      234
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      225
                     variables with only lower bounds:        0
                vari

  23  6.4494584e-03 1.13e-06 3.58e-01  -9.2 8.09e-01    -  1.89e-02 1.09e-03f  1
  24  2.2795918e-03 3.37e-02 5.72e+00  -9.5 8.04e-01    -  2.16e-02 1.00e+00f  1
  25  2.2672932e-03 3.36e-02 5.83e+00 -11.0 1.82e+00    -  1.66e-01 2.02e-03h  1
  26  1.5377890e-03 3.39e-02 5.83e+00  -7.1 2.47e+03    -  1.34e-04 9.64e-05h  1
  27  6.1630485e-04 3.64e-02 5.81e+00  -7.1 1.40e+03    -  3.17e-06 5.31e-04h  2
  28  6.0624547e-04 3.64e-02 5.73e+00  -8.2 6.20e+00    -  1.74e-01 7.42e-04h  1
  29  1.8383075e-04 3.40e-02 5.46e+00  -6.9 2.32e+00    -  6.48e-04 6.47e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.6786466e-04 3.40e-02 5.45e+00  -6.7 5.18e+01    -  2.97e-04 2.07e-03h  5
  31  1.6709351e-04 3.40e-02 5.45e+00  -6.7 6.57e+02    -  3.47e-03 4.07e-05h  3
  32  1.6681995e-04 3.40e-02 5.45e+00  -8.7 8.74e+01    -  8.96e-04 2.98e-04h  7
  33  1.6601110e-04 3.40e-02 5.45e+00  -8.7 2.63e+02    -  7.95e-03 1.03e-04h  3
  34  1.6607687e-04 3.26e-02